In [15]:
import numpy as np
import pandas as pd
import gurobipy as gp

from gurobipy import GRB

from pathlib import Path
import os
import sys
from datetime import datetime, date

In [16]:
result = Path('result/')
report = Path('report/')
instances = Path('instances/csifa')

In [17]:
file_name = sys.argv[1]

In [18]:
def read_instance(file_name):
	
	arq = open(file_name)
	
	N = int(arq.readline())
	
	PR  = np.zeros(N)
	PP  = np.zeros(N)

	FR = [float(arq.readline())]*N
	FP = [float(arq.readline())]*N

	HR = [float(arq.readline())]*N
	HP = [float(arq.readline())]*N

	D = [int(i) for i in arq.readline().split()]
	
	R = [int(i) for i in arq.readline().split()]

	C = float(arq.readline().rstrip('\n'))
	
	return N, PP, PR, FP, FR, HR, HP, D, R, C

In [19]:
MAX_CPU_TIME = 3600.0
EPSILON = 1e-6

def clsr_mc_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C):

	CP = [0]*N
	CR = [0]*N
	KP = 0
	KR = 0 

	for i in range(N):
		CP[i] = PP[i]

		for j in range(i,N):
			CP[i] = CP[i] + HP[j]

	for i in range(N):
		CR[i] = PR[i]
		for j in range(i,N):
			CR[i] += HP[j]
		for j in range(i,N):
			CR[i] -= HR[j]

	for i in range(N):
		AUX = 0 
		for j in range(i+1):
			AUX += D[j]

		KP += HP[i]*AUX

	for i in range(N):
		AUX = 0

		for j in range(i+1):
			AUX += R[j]

		KR += HR[i]*AUX

	K = KR - KP

	try:

		# Create a new model
		model = gp.Model("clsr_mc_mip")

		# Create variables
		wp = model.addVars([(i,j) for i in range(N) for j in range(i,N)], vtype=GRB.CONTINUOUS, name="wp")
		wr = model.addVars([(i,j) for i in range(N) for j in range(i,N)], vtype=GRB.CONTINUOUS, name="wr")
		vor = model.addVars([(i,j) for i in range(N) for j in range(i,N)], vtype=GRB.CONTINUOUS, name="vor")
		xp = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xp")
		xr = model.addVars(list(range(N)), vtype=GRB.CONTINUOUS, name="xr")
		yp = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yp")
		yr = model.addVars(list(range(N)), vtype=GRB.BINARY, name="yr")

		# # Set objective
		fobj = 0.0
		#fobj = gp.quicksum(xp[i]*CP[i] for i in range(N)) \
		# + gp.quicksum(yp[i]*FP[i] for i in range(N)) \
		# + gp.quicksum(xr[i]*CR[i] for i in range(N)) \
		# + gp.quicksum(yr[i]*FR[i] for i in range(N)) + K
		
		for i in range(N):
			fobj += xp[i]*CP[i]

		for i in range(N):
			fobj += yp[i]*FP[i]

		for i in range(N):
			fobj += xr[i]*CR[i]

		for i in range(N):
			fobj += yr[i]*FR[i]

		fobj += K
		
		model.setObjective(fobj, sense = GRB.MINIMIZE)

		for i in range(N):
			ctr = 0.0
			for j in range(i+1):
				ctr += wp[j,i]
				ctr += wr[j,i]
			model.addConstr(ctr >= D[i])

		for i in range(N):
			ctr = 0.0
			for j in range(i+1):
				ctr+= vor[j,i]
			for j in range(i,N):
				ctr+=(-wr[i,j])
			model.addConstr(ctr == 0)

		for i in range(N):
			ctr =0.0
			for j in range(i,N):
				ctr += vor[i,j]
			model.addConstr(ctr <= R[i])

		for i in range(N):
			for j in range(i,N):
				model.addConstr(wp[i,j] + yp[i]*(-D[j]) <= 0)

		for i in range(N):
			for j in range(i,N):
				model.addConstr(wr[i,j] + yr[i]*(-min(SR[0][i],D[j])) <= 0)

		for i in range(N):
			for j in range(i,N):
				model.addConstr(vor[i,j] + yr[j]*(-R[i]) <= 0)

		for i in range(N):
			ctr = 0.0 
			ctr += xp[i]
			for j in range(i,N):
				ctr += (-wp[i,j])
			model.addConstr(ctr == 0)

		for i in range(N):
			ctr = 0.0
			ctr += xr[i]
			for j in range(i,N):
				ctr += (-wr[i,j])
			model.addConstr(ctr == 0)

		#model.addConstrs(
		#	xp[i] - yp[i]*min(C,SD[i][N-1]) <= 0 for i in range(N)
		#	)

		#model.addConstrs(
		#	xr[i] - yr[i]*min(SR[0][i],SD[i][N-1],C) <= 0 for i in range(N)
		#	)

		model.addConstrs(
			xp[i] + xr[i] <= C for i in range(N)
			)

		# Parameters 
		model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
		model.setParam(GRB.Param.MIPGap, EPSILON)
		model.setParam(GRB.Param.Threads, 1)
		#model.setParam(GRB.Param.Cuts, -1)
		#model.setParam(GRB.Param.Presolve, -1)

		# Optimize model
		model.optimize()

		tmp=0
		if model.status == GRB.OPTIMAL:
			tmp=1

	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))

	return model.ObjVal, model.ObjBound, model.MIPGap, model.Runtime, model.NodeCount, tmp


In [20]:
def main(file_name):
	
	#file_name = f"c52_1.txt"

	N, PP, PR, FP, FR, HR, HP, D, R, C = read_instance(os.path.join(instances,file_name))

	SD = (np.zeros((N,N))).tolist()
	SR = (np.zeros((N,N))).tolist()

	for  i in range(N):
		SD[i][i] = D[i]
		SR[i][i] = R[i]
		for j in range(i+1,N):
			SD[i][j] = SD[i][j-1] + D[j]
			SR[i][j] = SR[i][j-1] + R[j]

	bestsol, bestbound, gap, runtime, numnode, status = clsr_sp_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C)
	
	arquivo = open(os.path.join(result,'clsr_sp_mip.txt'),'a')
	
	arquivo.write(file_name+';'+str(round(bestsol,2))+';'+str(round(bestbound,2))+\
					';'+str(round(gap,2))+';'+str(round(runtime,2))+';'+str(round(numnode,3))+\
					';'+str(round(status,2))+
					'\n')

	arquivo.close()

In [21]:
if __name__== "__main__" :

	file_name = f"c52_1.txt"

	main(file_name)

Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)



CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Optimize a model with 468 rows, 4290 columns and 20826 nonzeros
Model fingerprint: 0x92898b21
Variable types: 4186 continuous, 104 integer (104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [4e+00, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 18092.600000
Presolve removed 103 rows and 1 columns
Presolve time: 0.03s
Presolved: 365 rows, 4289 columns, 18016 nonzeros
Variable types: 4186 continuous, 103 integer (103 binary)

Root relaxation: objective 8.605767e+03, 701 iterations, 0.02 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8605.76650    0   77 18092.6000 8605.76650  52